In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from utils import *

In [3]:
path = untar_data(URLs.HUMAN_NUMBERS)
Path.BASE_PATH=path                
path.ls()

(#2) [Path('train.txt'),Path('valid.txt')]

In [4]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

(#9998) ['one \n','two \n','three \n','four \n','five \n','six \n','seven \n','eight \n','nine \n','ten \n'...]

In [5]:
text = ' . '.join([l.strip() for l in lines])
tokens = text.split(' ')
vocab = L(*tokens).unique()
word2idx={w:i for i,w in enumerate(vocab)} 
nums = L(word2idx[i] for i in tokens) 

In [6]:
bs=64
sl = 16
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))   #Check the 3 seqs to make this code
         for i in range(0,len(nums)-sl-1,sl))

m= len(seqs)//bs
m, bs, len(seqs)

def group_chunks(ds,bs):
    m = len(ds)//bs
    new_ds = L()
    for i in range(m): new_ds+= L(ds[i+m*j] for j in range(bs))
    return new_ds

cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [36]:
class LMModel(nn.Module):
    def __init__(self, vocab_sz, n_hidden):
        super(LMModel,self).__init__()
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.GRU(n_hidden, n_hidden,1,batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.bn = BatchNorm1dFlat(n_hidden)
        self.h = torch.zeros(1, bs, n_hidden)
        
    def forward(self, input):
        output, h = self.rnn(self.i_h(input), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(output))

In [37]:
learn = Learner(dls, LMModel(len(vocab),64), loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [38]:
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.496286,2.158223,0.402588,00:02
1,1.743994,1.863906,0.433268,00:01
2,1.189741,1.941908,0.544271,00:01
3,0.609933,1.255292,0.779948,00:01
4,0.296075,1.417631,0.799642,00:01
5,0.152147,1.535561,0.794759,00:01
6,0.085090,1.305259,0.814046,00:01
7,0.054443,1.368237,0.816813,00:01
8,0.039144,1.460885,0.805990,00:01
9,0.029461,1.396415,0.811117,00:01
